In [ ]:

import calendar
from datetime import datetime  # For working with dates

import matplotlib.pyplot as plt
import pandas as pd  # For working with DataFrames
import seaborn as sns
from PIL import Image
from matplotlib.ticker import FuncFormatter

logo_path = r'D:\Projects\BOT\reports\trash_media\logo.png'
logo = Image.open(logo_path)
size = 0.4
# Resize the logo to 43% of its original size
resized_width = int(logo.width * size)
resized_height = int(logo.height * size)
logo = logo.resize((resized_width, resized_height))

sample_data_path = r'trash_media/sample_data.csv'
promotion_path = r'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

CURRENT_MONTH = datetime.now().month
CURRENT_YEAR = datetime.now().year
_, days_in_month = calendar.monthrange(CURRENT_YEAR, CURRENT_MONTH)
date_begin = datetime(CURRENT_YEAR, CURRENT_MONTH, 1).strftime('%Y%m%d')
date_end = datetime(CURRENT_YEAR, CURRENT_MONTH, days_in_month).strftime('%Y%m%d')
df = pd.read_csv(sample_data_path)
df = df[df['DocumentType'].isin(['Оптовая реализация', 'Финансовая скидка'])
        & ~(df['Отпустил'].isin(['Бочкарева Альвина']))
        ]
df.sample(10)

In [ ]:
result_df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientManager', right_on='ClientMan',
                     how='left')

result_df['inn_temp'] = pd.to_numeric(result_df['INN'], errors='coerce')
types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
result_df = pd.merge(result_df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp',
                     how='left')
result_df.loc[result_df['TYPE'] == 'ROZ', 'RegionType'] = result_df['Region']
result_df['OXVAT'] = result_df['INN'].map(result_df['INN'].value_counts())
result_df.columns = ['DocName', 'InvoiceNumber', 'GoodId', 'Good', 'Producer', 'INN', 'ClientName', 'InvoiceManager',
                     'ClientManager', 'PaymentTerm', 'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered',
                     'BaseAmount', 'TotalAmount', 'ClientMan', 'Region', 'inn_temp', 'INN_temp', 'TYPE', 'RegionType',
                     'OXVAT']

categorical_columns = ['DocName', 'GoodId', 'Good', 'Producer', 'INN', 'ClientName',
                       'InvoiceManager', 'ClientManager', 'PaymentTerm', 'Region',
                       'RegionType', 'TYPE']

result_df[categorical_columns] = result_df[categorical_columns].astype('category')


In [ ]:
result_df.info()

In [ ]:
columns_to_drop = ([col for col in result_df.columns if col.endswith('_temp')] +
                   ['ClientMan'])
# 
# # Drop the identified columns
result_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
result_df.info()

In [ ]:
goods_totals = result_df.groupby(['ClientManager', 'Good'], observed=False)['Quantity'].sum().reset_index()
top_goods = goods_totals.sort_values(by='Quantity', ascending=False)

In [ ]:

# Function to format large numbers
def format_large_numbers(value, pos):
    if value >= 1e9:  # If the value is in billions
        return f'{value / 1e9:.1f}B'
    elif value >= 1e6:  # If the value is in millions
        return f'{value / 1e6:.1f}M'
    elif value >= 1e3:  # If the value is in thousands
        return f'{value / 1e3:.1f}K'
    else:  # For values less than 1000
        return str(int(value))


# Function to abbreviate or truncate the Good names
def abbreviate_good_name(name, max_words=3):
    words = name.split()
    if len(words) <= max_words:
        return name
    else:
        return ' '.join(words[:max_words]) + '...'



In [ ]:
def create_bar_plot(df, manager, title, x_column, y_column):
    print(f'Plotting {title}')
    # HOW MANY
    top_rows = 20
    df = df.head(top_rows).copy()
    relevant_columns = df.columns[3:]

    df[y_column] = df[y_column].apply(lambda x: abbreviate_good_name(x, max_words=3))

    # pastel_palette = sns.color_palette("tab20b")
    # output_picture_path = f'reports/trash_media/Top_20_Goods_{output_file_path.split("_")[1]}_{type_value}.png'

    # Create a new figure
    fig, ax = plt.subplots(figsize=(18, 10))
    # Plotting horizontal bars using Seaborn
    for m in manager:
        bars = sns.barplot(x=x_column, y=y_column, data=df[df['ClientManager'] == m], ax=ax,
                           errorbar=None, 
                           # hue=y_column,
                           )

        # Adding labels and title
        ax.set_xlabel(f'{x_column}')
        ax.set_ylabel(f'{y_column}')
        ax.set_title(f'{title} :{m}')
    
        # Annotating each bar with its value
        for bar, value in zip(bars.patches, df[x_column]):
            ax.text(bar.get_width() + 0.005 * bar.get_width(), bar.get_y() + bar.get_height() / 2,
                    format_large_numbers(value, None), va='center')
    
        # Format x-axis labels using the formatter function
        ax.xaxis.set_major_formatter(FuncFormatter(format_large_numbers))
        fig.figimage(logo, xo=1051.7, yo=65, alpha=0.5)  # Adjust xo, yo, and alpha as needed
        plt.savefig(f'Sales_of_{m}.png', bbox_inches='tight')
        plt.show()
        plt.close()
    


In [ ]:
import concurrent.futures

def plot_for_manager(df, manager, title, x_column, y_column):
    print(f'Plotting {title} for {manager}')
    df_copy = df.copy()
    df_copy[y_column] = df_copy[y_column].apply(lambda x: abbreviate_good_name(x, max_words=3))
    fig, ax = plt.subplots(figsize=(18, 10))
    bars = sns.barplot(x=x_column, y=y_column, data=df_copy[df_copy['ClientManager'] == manager], ax=ax,
                       errorbar=None)
    ax.set_xlabel(f'{x_column}')
    ax.set_ylabel(f'{y_column}')
    ax.set_title(f'{title} : {manager}')
    for bar, value in zip(bars.patches, df_copy[x_column]):
        ax.text(bar.get_width() + 0.005 * bar.get_width(), bar.get_y() + bar.get_height() / 2,
                format_large_numbers(value, None), va='center')
    ax.xaxis.set_major_formatter(FuncFormatter(format_large_numbers))
    fig.figimage(logo, xo=1051.7, yo=65, alpha=0.5)  # Adjust xo, yo, and alpha as needed
    plt.savefig(f'Sales_of_{manager}.png', bbox_inches='tight')
    plt.close()

def create_bar_plot_threaded(df, managers, title, x_column, y_column):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for manager in managers:
            futures.append(executor.submit(plot_for_manager, df, manager, title, x_column, y_column))
        for future in concurrent.futures.as_completed(futures):
            future.result()

# Example usage:
create_bar_plot_threaded(top_goods, managers=top_goods.ClientManager, title='Sales of', x_column='Quantity', y_column='Good')

In [ ]:
create_bar_plot(top_goods, manager=top_goods.ClientManager, title='Sales of', x_column='Quantity', y_column='Good')

In [ ]:
top_goods.info()